# Pandas: How to read the data files the right way

### Lets generate data_file that we would use to test performance

In [1]:
import pandas as pd
import time
import numpy as np
from scipy.sparse import csr_matrix # for creating sparse data

data_file = 'DELETE_ME_huge_file.csv' # Do NOT FORGET TO DELETE THIS AT THE END

C:\Users\hi\AppData\Local\Temp\ipykernel_5632\1893042939.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Lets generate some fake data and save to a csv file for testing:
# NOTE: Took 3 sec to generate the CSV file

n = 1_000_000 # Number of rows: 1 million

np.random.seed(42) # seed for reproducibility

# Generate data
data = {
    "id"     : np.arange(1, n + 1),
    "name"   : np.random.choice(["Fernando", "Prakash", "Shamlodhiya", "Smith", "Patel", "Juno"], size=n),
    "age"    : np.random.randint(20, 80, size=n),
    "amount" : np.round(np.random.uniform(50, 500, size=n), 2),
    "status" : np.random.choice(["Paid", "Pending", "Failed"], size=n, p=[0.6, 0.3, 0.1])
}

df = pd.DataFrame(data)

df.to_csv(data_file, index=False) # Save as CSV

print(f"Sample dataset '{data_file}' with {n} rows created successfully!")

Sample dataset 'DELETE_ME_huge_file.csv' with 1000000 rows created successfully!


In [26]:
# lets look at few values
print(df.sample(10))

            id         name  age  amount   status
758368  758369         Juno   76  128.56  Pending
537469  537470  Shamlodhiya   63  185.86  Pending
848667  848668        Smith   21  333.20     Paid
426683  426684  Shamlodhiya   56  134.34  Pending
128391  128392      Prakash   39  347.50     Paid
966893  966894         Juno   65  436.60     Paid
192879  192880     Fernando   57  438.13     Paid
530489  530490        Patel   54  454.59     Paid
847744  847745     Fernando   20  231.74     Paid
858672  858673     Fernando   30  413.73     Paid


## 1. Use dtype to specify column types
- Avoid pandas guessing data. **Define dtypes for each column during reading of file**
- The **process is slower but is memory efficient**...takes space in RAM

In [15]:
# step1: Without specifying dtypes
# NOTE: Took 0.4 secs

start = time.process_time()
df    = pd.read_csv(data_file)
end   = time.process_time()

print(df.head())
print("Time in sec:",end - start)
print(df.info(memory_usage="deep"))

   id         name  age  amount   status
0   1        Smith   78   58.66  Pending
1   2        Patel   71  207.34     Paid
2   3  Shamlodhiya   40   66.54     Paid
3   4        Patel   32  326.49  Pending
4   5        Patel   49  380.40     Paid
Time in sec: 0.390625
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 5 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   id      1000000 non-null  int64  
 1   name    1000000 non-null  object 
 2   age     1000000 non-null  int64  
 3   amount  1000000 non-null  float64
 4   status  1000000 non-null  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 142.8 MB
None


In [18]:
# step2: with dtype explicitly mentioned: 
# NOTE: Took 4.2 secs

dtypes = {
    'id'    : 'int32', 
    'name'  : 'category', # Note: if names repeat many times, then use 'category', else use 'string'
    'age'   : 'uint8',    # this num is going to be 0 - 120. So use uint8
    'amount': 'float32',
    'status': 'category'
}
start = time.process_time()
df = pd.read_csv(data_file, dtype=dtypes)
end = time.process_time()

print(df.head())
print("Time in sec:",end - start)
print(df.info(memory_usage="deep"))

   id         name  age      amount   status
0   1        Smith   78   58.660000  Pending
1   2        Patel   71  207.339996     Paid
2   3  Shamlodhiya   40   66.540001     Paid
3   4        Patel   32  326.489990  Pending
4   5        Patel   49  380.399994     Paid
Time in sec: 0.4375
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 5 columns):
 #   Column  Non-Null Count    Dtype   
---  ------  --------------    -----   
 0   id      1000000 non-null  int32   
 1   name    1000000 non-null  string  
 2   age     1000000 non-null  uint8   
 3   amount  1000000 non-null  float32 
 4   status  1000000 non-null  category
dtypes: category(1), float32(1), int32(1), string(1), uint8(1)
memory usage: 70.3 MB
None


In [6]:
# How much memory saved? 92%
(143-11)/143

0.9230769230769231

## conclusion: The process slows down little bit, but you save memory

## 2. Load only required columns: 
- Make use of **usecols**

In [7]:
# step1: Read all columns: The hard way

start = time.process_time()
df = pd.read_csv(data_file)
end = time.process_time()

print(df.head())
print(end - start)
print(df.info(memory_usage="deep"))

   id         name  age  amount   status
0   1        Smith   78   58.66  Pending
1   2        Patel   71  207.34     Paid
2   3  Shamlodhiya   40   66.54     Paid
3   4        Patel   32  326.49  Pending
4   5        Patel   49  380.40     Paid
0.296875
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 5 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   id      1000000 non-null  int64  
 1   name    1000000 non-null  object 
 2   age     1000000 non-null  int64  
 3   amount  1000000 non-null  float64
 4   status  1000000 non-null  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 142.8 MB
None


In [8]:
# step2: Read only required columns

start = time.process_time()
df = pd.read_csv(data_file, usecols=['age', 'amount'])
end = time.process_time()

print(df.head())
print(end - start)
print(df.info(memory_usage="deep"))

   age  amount
0   78   58.66
1   71  207.34
2   40   66.54
3   32  326.49
4   49  380.40
0.3125
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   age     1000000 non-null  int64  
 1   amount  1000000 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 15.3 MB
None


In [9]:
# How much saving in memory: 89%
(142-15.3)/142

0.8922535211267606

### 3. Sparse data
- Most values are the same (often zeros or NaNs) and only a small fraction are **useful** non-zero values.

In [19]:
# step1: Lets generate some fake data and save to a csv file for testing:

n = 100000 # Number of rows

np.random.seed(42) # seed for reproducibility

sparsity = 0.9  
values = np.random.randint(1, 10, size=n) # Random values
mask = np.random.choice([0, 1], size=n, p=[sparsity, 1-sparsity]) # Mask for sparsity
sparse_data = values * mask
sparse_vec = csr_matrix(sparse_data) # Convert to sparse vector

# Generate data
data = {
    "id"          : np.arange(1, n + 1),
    "commute_dist": sparse_vec.toarray().ravel()
}

df = pd.DataFrame(data)

df.to_csv(data_file, index=False) # Save as CSV

print(f"Sample dataset '{data_file}' with {n} rows created successfully!")

Sample dataset 'DELETE_ME_huge_file.csv' with 100000 rows created successfully!


In [22]:
# Lets see the data for column commute_dist
print(df.sample(10))
print(df['commute_dist'].value_counts())

          id  commute_dist
10761  10762             0
32648  32649             0
92274  92275             0
19648  19649             0
12561  12562             0
72694  72695             0
18386  18387             0
84271  84272             0
73197  73198             0
65941  65942             0
commute_dist
0    90031
6     1147
2     1143
7     1127
8     1118
5     1100
9     1097
4     1087
3     1081
1     1069
Name: count, dtype: int64


In [20]:
# step2: read the CSV file the hard way: Without specifying sparse datatype

df = pd.read_csv(data_file)

print("Dense DataFrame memory usage:")
print(df.memory_usage(deep=True))
print("Total:", df.memory_usage(deep=True).sum(), "bytes")

Dense DataFrame memory usage:
Index              128
id              800000
commute_dist    800000
dtype: int64
Total: 1600128 bytes


In [12]:
# step3: convert to pandas SparseDtype

df_sparse = df.copy()
df_sparse["commute_dist"] = df_sparse["commute_dist"].astype("Sparse[int]")

print("\nSparse DataFrame memory usage:")
print(df_sparse.memory_usage(deep=True))
print("Total:", df_sparse.memory_usage(deep=True).sum(), "bytes")


Sparse DataFrame memory usage:
Index              128
id              800000
commute_dist     79752
dtype: int64
Total: 879880 bytes


In [13]:
# savings : 90%
(800000-79752)/800000

0.90031

In [ ]:
######################################

## Load Boolean type
- If we have columns that are boolean in nature, then use dataype boolean

In [25]:
# Dealing boolean data type
# step1: Lets generate some fake data and save to a csv file for testing:

n = 1_000_000 # Number of rows: 1 million

np.random.seed(42) # seed for reproducibility

# Generate data
data = {
    "employee_id" : np.arange(1, n + 1),
    "is_working"  : np.random.choice(["Not working", "Working"], size=n), # only 2 string choice
    "is_remote"  : np.random.choice([0, 1], p=[0.9, 0.1], size=n),
}

df = pd.DataFrame(data)
df.to_csv(data_file, index=False) # Save as CSV
print(f"Sample dataset '{data_file}' with {n} rows created successfully!")

Sample dataset 'DELETE_ME_huge_file.csv' with 1000000 rows created successfully!


In [26]:
# lets see few records
print(df.head())

   employee_id   is_working  is_remote
0            1  Not working          0
1            2      Working          0
2            3  Not working          0
3            4  Not working          1
4            5  Not working          0


In [27]:
# step2: load file without dtypes

df = pd.read_csv(data_file)

# print memory stats
print(df.info(memory_usage="deep"),"\n")
print(df.memory_usage(deep=True),"\n")
print("Total:", df.memory_usage(deep=True).sum(), "bytes")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   employee_id  1000000 non-null  int64 
 1   is_working   1000000 non-null  object
 2   is_remote    1000000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 70.6 MB
None 

Index               132
employee_id     8000000
is_working     58000756
is_remote       8000000
dtype: int64 

Total: 74000888 bytes


In [28]:
# step3: Explicitly set dtypes
dtypes = {
    "employee_id": "int32",      # 1M fits in int32 (max 2.1B)
    "is_working": "category",    # only 2 unique values → best as category
    "is_remote": "bool"          # NOTE: use 'bool' if column has no missing values, else use 'boolean'
}

df = pd.read_csv(data_file, dtype=dtypes)

# print memory stats
print(df.info(memory_usage="deep"),"\n")
print(df.memory_usage(deep=True),"\n")
print("Total:", df.memory_usage(deep=True).sum(), "bytes")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype   
---  ------       --------------    -----   
 0   employee_id  1000000 non-null  int32   
 1   is_working   1000000 non-null  category
 2   is_remote    1000000 non-null  bool    
dtypes: bool(1), category(1), int32(1)
memory usage: 5.7 MB
None 

Index              132
employee_id    4000000
is_working     1000116
is_remote      1000000
dtype: int64 

Total: 6000248 bytes


In [29]:
# Lets look at few rows
print(df.head())

   employee_id   is_working  is_remote
0            1  Not working      False
1            2      Working      False
2            3  Not working      False
3            4  Not working       True
4            5  Not working      False


In [30]:
# how much saving for column is_working: 98%
print((58000756 -  1000116)/58000756)

# how much saving for column is_remote: 87%
print((8000000 - 1000000)/8000000)

0.9827568454452559
0.875


In [32]:
# step4: (OPTIONAL)  for column is_working, map "Yes"->True, "No"->False and change the dtype to bool

# Convert Yes/No → True/False
df["is_working"] = df["is_working"].map({"Working": True, "Not working": False}).astype("bool")

print(df.info(memory_usage="deep"),"\n")
print(df.memory_usage(deep=True),"\n")
print("Total:", df.memory_usage(deep=True).sum(), "bytes")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype
---  ------       --------------    -----
 0   employee_id  1000000 non-null  int32
 1   is_working   1000000 non-null  bool 
 2   is_remote    1000000 non-null  bool 
dtypes: bool(2), int32(1)
memory usage: 5.7 MB
None 

Index              132
employee_id    4000000
is_working     1000000
is_remote      1000000
dtype: int64 

Total: 6000132 bytes


In [ ]:
# How much saving for is_working: 0.01% (Not significant here)
(1000116 - 1000000)/1000116

0.00011598654556071496

In [ ]:
################

## 4. Process large files in chunks using chunksize

### When to use chunking ?

You only need to aggregate results (e.g., sum, mean, counts).

In [6]:
# 1. Create a large CSV (5 million rows) # took 30 sec
rows = 50_000_000
df = pd.DataFrame({
    "id": np.arange(rows),
    "value": np.random.randint(0, 100, size=rows)
})

df.to_csv(data_file, index=False)

In [10]:
# 2. Read at once (memory heavy)
start   = time.process_time()
df_full = pd.read_csv(data_file)
total_sum_full = df_full["value"].sum()
end     = time.process_time()
print(f"Full load:    sum={total_sum_full}, time={end - start:.2f} sec")

Full load:    sum=2475237176, time=4.78 sec


In [11]:
# 3. Read in chunks (memory light)
start = time.process_time()
total_sum_chunk = 0
chunks = pd.read_csv(data_file, chunksize=100_000)  # 100k rows at a time
for chunk in chunks:
    total_sum_chunk += chunk["value"].sum()
end = time.process_time()
print(f"Chunked load: sum={total_sum_chunk}, time={end - start:.2f} sec")

Chunked load: sum=2475237176, time=7.31 sec


## conclusion
- Both methods give the same result
- Chunking uses far less memory (you’re only holding 100k rows in memory at any given time instead of 5 million).
- The speed may be slightly better or slightly worse depending on system I/O, but the **real win is memory efficiency**.

In [ ]:
############################

In [19]:
# Now lets calculate mean value using chunking: took 12 sec

start = time.process_time()

total_sum   = 0
total_count = 0

chunks = pd.read_csv(data_file, chunksize=100_000)  # 100k rows at a time
for chunk in chunks:
    total_sum   += chunk["value"].sum()
    total_count += chunk["value"].count()   # count non-NaN values

mean_value = total_sum / total_count

end = time.process_time()

print(f"Chunked load: mean={mean_value}, time={end - start:.2f} sec")

Chunked load: mean=49.49528002, time=6.75 sec


## Use compression if reading from zipped files
### Why compression helps

- Smaller file size on disk and reduces storage costs
- A 1 GB CSV can shrink to ~200 MB (or less) with gzip.
- Faster reads (despite decompression)
- Reading from disk/network is often the slowest part (I/O bottleneck). **If the file is compressed, you transfer/read fewer bytes.**

## Test it if it improves performance. In my case it did NOT
- 

In [12]:
import os

# step 1) generate fake data. (Note: This took 10 sec)
n = 1_000_000 # Generate dataset
np.random.seed(42)

data = {
    "employee_id": np.arange(1, n + 1),
    "is_working" : np.random.choice(["Yes", "No"], size=n),
    "is_remote"  : np.random.choice([0, 1], p=[0.9, 0.1], size=n),
    "value"      : np.random.randn(n) * 100  # extra numeric column
}

df = pd.DataFrame(data)

csv_file = "DELETE_big_data.csv" # Save uncompressed CSV
df.to_csv(csv_file, index=False)

csv_gz_file = "DELETE_big_data.csv.gz" # Save compressed CSV (gzip)
df.to_csv(csv_gz_file, index=False, compression="gzip")

In [13]:
# step2) Compare file sizes

size_uncompressed = os.path.getsize(csv_file) / (1024**2)
size_compressed = os.path.getsize(csv_gz_file) / (1024**2)
print(f"File size (uncompressed): {size_uncompressed:.2f} MB")
print(f"File size (gzip):         {size_compressed:.2f} MB")

File size (uncompressed): 30.78 MB
File size (gzip):         12.20 MB


In [16]:
# 3. Benchmark read times + memory

start = time.process_time()
df = pd.read_csv(csv_file)
end = time.process_time()
mem = df.memory_usage(deep=True).sum() / (1024**2)
print(f"Read {csv_file:<15} | Time: {end - start:.2f} sec | Memory: {mem:.2f} MB")


start = time.process_time()
df = pd.read_csv(DELETE_big_data, compression="gzip")
end = time.process_time()
mem = df.memory_usage(deep=True).sum() / (1024**2)
print(f"Read {csv_gz_file:<15} | Time: {end - start:.2f} sec | Memory: {mem:.2f} MB")

Read DELETE_big_data.csv | Time: 0.28 sec | Memory: 79.63 MB
Read DELETE_big_data.csv.gz | Time: 0.56 sec | Memory: 79.63 MB


In [ ]:
# ## 3. Parse dates efficiently using parse_dates
df = pd.read_csv(data_file, parse_dates=['created_at'])

In [ ]:
# STOP (Delete the data_file)

In [48]:
# ## 8. Use faster backend engines (pandas 2.0+)
# df = pd.read_csv(data_file, engine='pyarrow')  # Alternative to engine='c'

ImportError: Missing optional dependency 'pyarrow'.  Use pip or conda to install pyarrow.